In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [1]:
!git clone https://github.com/minjae0501/yolo_block.git

Cloning into 'yolo_block'...
remote: Enumerating objects: 1282, done.
remote: Counting objects: 100% (1282/1282), done.
remote: Compressing objects: 100% (968/968), done.
remote: Total 1282 (delta 11), reused 1278 (delta 10), pack-reused 0
Receiving objects: 100% (1282/1282), 30.41 MiB | 23.38 MiB/s, done.
Resolving deltas: 100% (11/11), done.


In [2]:
# BASIC CONFIG (필요한 파라미터 값 여기서 설정)

data_dir = '/content/yolo_block/data/truck_step/'
model = 'yolov8n.pt'  # pretrained yolov8n

classes = ['truck_step_01', 'truck_step_02', 'truck_step_03', 'truck_step_04', 'truck_step_05']  # 라벨명
yaml_file = 'block_data.yaml'  # yaml 파일명

training_epoch = 100
patience = 30  # 지정한 epoch 만큼 성능이 개선이 안되면 조기종료.
batch = 32  # batch size (dafault)
imgsz = 640  # input batch size (default)
optimizer = 'Adam'


In [3]:
!nvidia-smi

Sat Dec 30 15:03:01 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   45C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

### ULTRALYTICS SETUP

In [4]:
%pip install ultralytics
import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.0.231 🚀 Python-3.10.12 torch-2.1.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
Setup complete ✅ (8 CPUs, 51.0 GB RAM, 26.3/166.8 GB disk)


In [5]:
from ultralytics import YOLO
import os
import yaml
from IPython.display import display, Image
from IPython import display

display.clear_output()
!yolo checks

Ultralytics YOLOv8.0.231 🚀 Python-3.10.12 torch-2.1.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
Setup complete ✅ (8 CPUs, 51.0 GB RAM, 26.3/166.8 GB disk)

OS                  Linux-6.1.58+-x86_64-with-glibc2.35
Environment         Colab
Python              3.10.12
Install             pip
RAM                 50.99 GB
CPU                 Intel Xeon 2.20GHz
CUDA                12.1

matplotlib          ✅ 3.7.1>=3.3.0
numpy               ✅ 1.23.5>=1.22.2
opencv-python       ✅ 4.8.0.76>=4.6.0
pillow              ✅ 9.4.0>=7.1.2
pyyaml              ✅ 6.0.1>=5.3.1
requests            ✅ 2.31.0>=2.23.0
scipy               ✅ 1.11.4>=1.4.1
torch               ✅ 2.1.0+cu121>=1.8.0
torchvision         ✅ 0.16.0+cu121>=0.9.0
tqdm                ✅ 4.66.1>=4.64.0
psutil              ✅ 5.9.5
py-cpuinfo          ✅ 9.0.0
thop                ✅ 0.1.1-2209072238>=0.1.1
pandas              ✅ 1.5.3>=1.1.4
seaborn             ✅ 0.12.2>=0.11.0


### Create yaml file

In [6]:
# python: dictionary <----------> yaml
data = {
    'train': data_dir + 'train/',
    'val': data_dir + 'val/',
    'test': data_dir + 'test/',
    'nc': 5,
    'names': classes
}

os.makedirs(data_dir, exist_ok = True) # directory to save

with open(data_dir + yaml_file, 'wt') as fw:
    yaml.dump(data, fw)


In [7]:
with open(data_dir + yaml_file, 'rt') as fr:
    d = yaml.safe_load(fr)
    print(type(d))
    print(d)

<class 'dict'>
{'names': ['truck_step_01', 'truck_step_02', 'truck_step_03', 'truck_step_04', 'truck_step_05'], 'nc': 5, 'test': '/content/yolo_block/data/truck_step/test/', 'train': '/content/yolo_block/data/truck_step/train/', 'val': '/content/yolo_block/data/truck_step/val/'}


### Model Training

In [8]:
import torch

if torch.cuda.is_available() == True:
    device = 'cuda:0'
else:
    device = 'cpu'

print(device)

cuda:0


In [9]:
model = YOLO(model).to(device)  # pretrained yolov8n

model.train(data = data_dir + yaml_file,
            epochs = training_epoch,
            patience = patience, # 지정한 epoch 만큼 성능이 개선이 안되면 조기종료.
            batch = batch, # batch size (dafault)
            imgsz = imgsz, # input batch size (default)
            optimizer = optimizer)

100%|██████████| 6.23M/6.23M [00:00<00:00, 25.2MB/s]


engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/content/yolo_block/data/truck_step/block_data.yaml, epochs=100, time=None, patience=30, batch=32, imgsz=640, save=True, save_period=-1, cache=False, device=cuda:0, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=Adam, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=True, line_width=None, format=torchscript, keras=False, optimize=False, i

100%|██████████| 755k/755k [00:00<00:00, 5.11MB/s]


Overriding model.yaml nc=80 with nc=5

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics

train: Scanning /content/yolo_block/data/truck_step/train/labels... 240 images, 0 backgrounds, 0 corrupt: 100%|██████████| 240/240 [00:00<00:00, 1260.98it/s]

...

                   all         60         60      0.981          1      0.995       0.85



100 epochs completed in 0.081 hours.
Optimizer stripped from runs/detect/train/weights/last.pt, 6.3MB
Optimizer stripped from runs/detect/train/weights/best.pt, 6.3MB

Validating runs/detect/train/weights/best.pt...
Ultralytics YOLOv8.0.231 🚀 Python-3.10.12 torch-2.1.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 3006623 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  2.47it/s]


                   all         60         60      0.981          1      0.995      0.849
         truck_step_01         60         12      0.996          1      0.995      0.817
         truck_step_02         60          8      0.974          1      0.995      0.724
         truck_step_03         60         20      0.988          1      0.995      0.906
         truck_step_04         60         16      0.983          1      0.995      0.903
         truck_step_05         60          4      0.964          1      0.995      0.896
Speed: 0.2ms preprocess, 2.3ms inference, 0.0ms loss, 0.7ms postprocess per image
Results saved to runs/detect/train


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3, 4])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7dfded5d2f20>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
        

### Model Evaluation

In [10]:
best_model_path = '/content/runs/detect/train/weights/best.pt'

best_model = YOLO(best_model_path).to(device)
metrics = best_model.val()

for label, ap in zip(classes, metrics.box.maps):
    print(label, ':', ap)

Model summary (fused): 168 layers, 3006623 parameters, 0 gradients, 8.1 GFLOPs


val: Scanning /content/yolo_block/data/truck_step/val/labels.cache... 60 images, 0 backgrounds, 0 corrupt: 100%|██████████| 60/60 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:02<00:00,  1.79it/s]


                   all         60         60      0.981          1      0.995      0.849
         truck_step_01         60         12      0.996          1      0.995      0.818
         truck_step_02         60          8      0.974          1      0.995      0.724
         truck_step_03         60         20      0.988          1      0.995      0.906
         truck_step_04         60         16      0.983          1      0.995      0.902
         truck_step_05         60          4      0.964          1      0.995      0.896
Speed: 0.3ms preprocess, 9.5ms inference, 0.0ms loss, 15.1ms postprocess per image
Results saved to runs/detect/val
truck_step_01 : 0.8178705964912281
truck_step_02 : 0.7237658666666665
truck_step_03 : 0.9064789969834088
truck_step_04 : 0.9023271566998892
truck_step_05 : 0.8964782608695652
